### Tools Integration Testing

This is used to test our tools that we plan to integrate with agents for callinng the Regulations.gov API.


In [1]:
# -----------------------------------------------------------------------------------------------------
# Setup the notebook to import from the `app` directory
# -----------------------------------------------------------------------------------------------------
import sys
import os
import requests
# Get the parent directory of `app` (the directory our notebook is in) and add it to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Notify the user that the parent directory has been added to sys.path
# We can now import the tools from the `app` directory
print(f"Absolute imports enabled. Parent directory added to sys.path: {parent_dir}")


# -----------------------------------------------------------------------------------------------------
# Create utility functions for displaying JSON responses
# -----------------------------------------------------------------------------------------------------
import json
from IPython.display import display, HTML

def pretty_print_json(data):
    """
    Pretty prints JSON data in a scrolling region in a Jupyter Notebook.

    Args:
        data (dict): The JSON data to pretty print.
    """
    # Convert the JSON data to a formatted string
    pretty_json = json.dumps(data, indent=4)
    
    # Create an HTML block with a scrolling region
    html_content = f"""
    <div style="overflow: auto; height: 300px; border: 1px solid #ccc; padding: 10px; font-family: monospace;">
        <pre>{pretty_json}</pre>
    </div>
    """
    display(HTML(html_content))


def pretty_print_text(text):
    """
    Pretty prints text in a scrolling region in a Jupyter Notebook.

    Args:
        text (str): The text to pretty print.
    """
    display(HTML(f"<div style='overflow: auto; height: 300px; border: 1px solid #ccc; padding: 10px; font-family: monospace;'>{text}</div>"))


Absolute imports enabled. Parent directory added to sys.path: /Users/kevintupper/source/agent_workshop


### Import tools for testing

In [2]:
# -----------------------------------------------------------------------------------------------------
# Import the tools
# -----------------------------------------------------------------------------------------------------
from app.rga_tools import get_documents, get_document_details, get_comments, get_comment_details, get_dockets, get_docket_details


### Test get_documents

In [3]:
# -----------------------------------------------------------------------------------------------------
# Test the get_documents tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_documents(agencyId="DOS", searchTerm="ITAR", pageNumber=1, pageSize=5)
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")

# Convert the JSON response to a DocumentFindAllResponse object
from app.rga_types import DocumentFindAllResponse
response_object = DocumentFindAllResponse.model_validate(response)

# Access the parsed object and print the number of elements as a string response.
print(f"Total elements (documents) meeting the search criteria: {response_object.meta.totalElements}")


# Get the first object id (id of the document), and the docketId for the next test.
first_document_id = response_object.data[0].id
first_docket_id = response_object.data[0].attributes.docketId
first_title = response_object.data[0].attributes.title

# Print the first document in the list
print("\nFirst Document in list:\n")
print(f"    Title: {first_title}")
print(f"    Document ID: {first_document_id}")
print(f"    Docket ID: {first_docket_id}")



Total elements (documents) meeting the search criteria: 313

First Document in list:

    Title: International Traffic & Arms Regulation (ITAR): Consolidation & Restructuring of Purpose & Definitions
    Document ID: DOS-2022-0004-0002
    Docket ID: DOS-2022-0004


### Test get_document_details

In [4]:
# -----------------------------------------------------------------------------------------------------
# Test the get_document_details tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_document_details(document_id=first_document_id)
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")

# Convert the JSON response to a DocumentFindOneResponse object
from app.rga_types import DocumentFindOneResponse
response_object = DocumentFindOneResponse.model_validate(response)

# Print the title of the document
print(f"Title of the document: {response_object.data.attributes.title}")

# Print the abstract of the document
print(f"Abstract of the document: {response_object.data.attributes.docAbstract}")

# Print the file url of the document
print(f"File URL of the document: {response_object.data.attributes.fileFormats[0].fileUrl}")
file_url = response_object.data.attributes.fileFormats[0].fileUrl



Title of the document: International Traffic & Arms Regulation (ITAR): Consolidation & Restructuring of Purpose & Definitions
Abstract of the document: None
File URL of the document: https://downloads.regulations.gov/DOS-2022-0004-0002/content.pdf


### Test get_comments

In [5]:
# -----------------------------------------------------------------------------------------------------
# Test the get_comments tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_comments(agencyId="DOS", searchTerm="ITAR", pageNumber=1, pageSize=5, sort="-postedDate")
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")


# Create comment_id variable
comment_id = None

# Get the first comment where a documentType is "Public Submission"
for comment in response['data']:
    if comment['attributes']['documentType'] == "Public Submission":
        comment_id = comment['id']
        print(f"Comment ID: {comment_id} is a Public Submission")
        break

Comment ID: DOS-2024-0035-0685 is a Public Submission


### Test get_comment_details

In [6]:
# -----------------------------------------------------------------------------------------------------
# Test the get_comment_details tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_comment_details(comment_id=comment_id, include_attachments=True)
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")


# Get the text of the comment
pretty_print_text(response['data']['attributes']['comment'])



### Test get_dockets

In [7]:
# -----------------------------------------------------------------------------------------------------
# Test the get_dockets tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_dockets(agencyId="DOS", searchTerm="ITAR", pageNumber=1, pageSize=5, sort="-lastModifiedDate")
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")

# Get the first docket id
first_docket_id = response['data'][0]['id']
print(f"First docket ID: {first_docket_id}")



First docket ID: DOS-2024-0035


### Test get_docket_details

In [9]:
# -----------------------------------------------------------------------------------------------------
# Test the get_docket_details tool
# -----------------------------------------------------------------------------------------------------

# Make the call to the API and pretty print the response
try:
    response = get_docket_details(docketId=first_docket_id)
    pretty_print_json(response)
except requests.RequestException as e:
    print(f"Error: {e}")